In [1]:
import pandas as pd
import os

In [2]:
station_id_table = {
    '臺北':'46692',
    '嘉義':'46748',
    '高雄':'46744',
    '東吉島':'46730',
    '臺西':'C0K53',
    '臺中電廠':'C0FA4'
}

In [3]:
final_col_list = ['觀測時間(day)',
                 '氣溫(℃)',
                 '最高氣溫(℃)',
                 '最低氣溫(℃)',
                 '相對溼度(%)',
                 '風速(m/s)',
                 '風向(360degree)',
                 '最大瞬間風(m/s)',
                 '最大瞬間風風向(360degree)',
                 '降水量(mm)',
                 '降水時數(hour)',
                 '日照時數(hour)',
                 '日照率(%)',
                 '全天空日射量(MJ/㎡)',
                 '總雲量(0~10)'
                 ]

In [31]:
start_time = '2023-01'
raw_data_path = '../data/weather/raw/'
raw_filenames = os.listdir('../data/weather/raw/')
destination_filename = '../data/weather/finalized/big_table.csv'

In [5]:
if os.path.exists(destination_filenames):
    final_df = pd.read_csv(destination_filenames)

In [6]:
def check_date_valid(start_time, filename):
    if int(start_time[0:4]) < int(filename.split('-')[1]):
        return True
    if int(start_time[0:4]) > int(filename.split('-')[1]):
        return False
    if int(start_time[0:4]) == int(filename.split('-')[1]):
        if int(start_time[6:8]) <= int(filename.split('-')[2].split('.')[0]):
            return True
        else:
            return False

In [7]:
for station_name, station_id in station_id_table.items():
    this_fname_list = [raw_data_path + fn for fn in raw_filenames if station_id in fn and check_date_valid(start_time, fn)]
    for fn in this_fname_list:
        yr_month_str = fn.split('.')[-2][-7::].replace('-', '/') + '/'
        this_df = pd.read_csv(fn).loc[1::]
        this_col_name_list = [col for col in this_df.columns if col in final_col_list]
        this_df = this_df[this_col_name_list]
        this_df.insert(0, '站名', [station_name] * len(this_df))
        this_df['觀測時間(day)'] = [yr_month_str + s for s in this_df['觀測時間(day)']]

        if 'final_df' in globals():
            final_df = pd.concat([final_df, this_df], axis=0, ignore_index=True)
        else:
            final_df = this_df

In [33]:
final_df.to_csv(destination_filename, encoding='utf-8-sig')

In [8]:
final_df

,站名,觀測時間(day),氣溫(℃),最高氣溫(℃),最低氣溫(℃),相對溼度(%),風速(m/s),風向(360degree),最大瞬間風(m/s),最大瞬間風風向(360degree),降水量(mm),降水時數(hour),日照時數(hour),日照率(%),全天空日射量(MJ/㎡),總雲量(0~10)
0,臺北,2023/01/01,19.3,22.0,16.8,83,3.2,90,12.5,100,1.5,2.6,0.5,4.7,8.98,9.0
1,臺北,2023/01/02,18.5,20.4,16.6,89,2.6,90,11.1,90,2.0,6.6,0.0,0.0,4.00,9.8
2,臺北,2023/01/03,16.1,17.7,14.4,89,3.0,90,12.6,120,T,2.3,0.0,0.0,7.10,10.0
3,臺北,2023/01/04,16.3,17.9,14.5,90,3.1,90,11.0,60,T,5.9,0.2,1.9,6.24,9.0
4,臺北,2023/01/05,19.3,22.2,17.2,90,2.0,90,9.3,90,1.0,2.8,1.3,12.3,9.25,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,臺中電廠,2024/07/03,30.7,34.9,27.5,67,1.6,170,7.6,348,0.0,NaN,NaN,NaN,NaN,NaN
3308,臺中電廠,2024/07/04,30.6,33.2,28.1,74,1.5,334,7.4,356,0.0,NaN,NaN,NaN,NaN,NaN
3309,臺中電廠,2024/07/05,30.3,32.4,28.1,77,1.9,336,8.3,331,0.0,NaN,NaN,NaN,NaN,NaN
3310,臺中電廠,2024/07/06,29.9,33.0,24.9,75,1.6,153,9.7,157,0.0,NaN,NaN,NaN,NaN,NaN


import matplotlib
import matplotlib.pyplot as plt
import datetime
matplotlib.rc('font', family='Microsoft JhengHei')

def alt_float(s):
    try:
        return float(s)
    except:
        pass

fig, ax = plt.subplots()
for station_name in station_id_table.keys():
    this_y = [alt_float(s) for s in final_df[final_df['站名']==station_name]['風速(m/s)']]
    this_x = [datetime.datetime.strptime(s, '%Y/%m/%d').date() for s in final_df[final_df['站名']==station_name]['觀測時間(day)']]
    ax.plot(this_x, this_y, label=station_name)
ax.legend()
#plt.plot([float(s) for s in final_df[final_df['站名']=='東吉島']['風速(m/s)']])